In [1]:
from Gemini.GeminiPredict import GeminiPredict
from Generation.Equation_BaseDataset import Equation, BaseDataset
from Postprocessing.SOE_EquationGraph import SystemOfEquations, EquationGraph
from Scraping.Scraper import Scraper

import os
import google.generativeai as genai
from sympy import *

### Gemini

In [2]:
api_key = os.getenv("GEMINI_API_KEY")
gp = GeminiPredict(api_key, "Data/HTC_equations.txt", "tunedModels/mmltopython4-f3fuppiemnq9")
gp.predict_from_txt()
gp.save_predictions("Data/HTC_gemini_predictions.txt")

API key loaded successfully!


Generating Predictions: 100%|██████████| 13/13 [00:19<00:00,  1.46s/row]


In [3]:
gp.save_predictions()
# for model in genai.list_tuned_models():
#     print(model.name)

### Generation

In [ ]:
eq = Equation()


In [4]:
class GeminiDataset(BaseDataset):
    def __init__(self, num, filepath):
        super().__init__(num, filepath)

    def get_columns(self):
        return ["text_input", "output"]
    
gd = GeminiDataset(50, "Data/gemini_test_4.csv")
gd.create()

Generating dataset: 100%|██████████| 50/50 [00:00<00:00, 66.17it/s]


### Postprocessing

In [2]:
soe = SystemOfEquations("Data/HTC_gemini5_predictions.txt")
for eq in soe.str_equations:
    soe.parse_equation(eq)
print(soe.eq)

[Eq(h, h_c + h_g), Eq(h, 1.45*k*(p/H)**0.985*tan(θ)/σ), Eq(h, 8000*λ*(K*p/(C*σ_U))**0.86), Eq(h, A*(1 - exp(-B*P))), Eq(h, 2*k_f*k_t*k_w*(1 - A)/(h_f*(-k_f*k_t - k_f*k_w + 2*k_t*k_w))), Eq(h, h_a + h_c + h_l), Eq(h_c, K_st*N_P*α/R), Eq(K_st, 2/(1/k_t + 1/k_s)), Eq(R, sqrt(Rs**2 + Rt**2)), Eq(N_P, 1 - exp(-P*λ/σ_U)), Eq(h_l, K_stl*N_L*β/R), Eq(K_stl, 3/(1/k_t + 1/k_s + 1/k_l)), Eq(N_L, 1 - exp(-γ*δ))]


In [3]:
# Constants
k_s = 0.14
k_t = 0.0315
k_l = 0.024
Rs = 3.4e-07
Rt = 9.6e-07
h_a = 0.8
σ_U = 21.0
α = 0.000201
λ = 6.05
β = 0.00011
γ = 200000.0
δ = 1.5e-5
P = 10

k = 1
p = 1
H = 1
θ = 1
σ = 1
K = 1
C = 1
A = 1
B = 1
k_f = 1
k_w = 1
h_f = 1
h_g = 1

# Variables
h = Symbol("h")
h_c = Symbol("h_c")
K_st = Symbol("K_st")
R = Symbol("R")
N_P = Symbol("N_P")
h_l = Symbol("h_l")
K_stl = Symbol("K_stl")
N_L = Symbol("N_L")
a = Symbol("a")
b = Symbol("b")

exprs = [Eq(h, h_c + h_g), 
            Eq(h, 1.45*k*(p/H)**0.985*tan(θ)/σ), 
            Eq(h, 8000*λ*(K*p/(C*σ_U))**0.86), 
            Eq(h, A*(1 - exp(-B*P))), 
            Eq(h, 2*k_f*k_t*k_w*(1 - A)/(h_f*(-k_f*k_t - k_f*k_w + 2*k_t*k_w))), 
            Eq(h, h_a + h_c + h_l), 
            Eq(h_c, K_st*N_P*α/R), 
            Eq(K_st, 2/(1/k_t + 1/k_s)), 
            Eq(R, sqrt(Rs**2 + Rt**2)), 
            Eq(N_P, 1 - exp(-P*λ/σ_U)), 
            Eq(h_l, K_stl*N_L*β/R), 
            Eq(K_stl, 3/(1/k_t + 1/k_s + 1/k_l)), 
            Eq(N_L, 1 - exp(-γ*δ))]


eg = EquationGraph(exprs, 6)
reduced = eg.get_system_of_equations()
sol1 = solve(reduced)[0]
sol2 = solve(eg.equations)[0]
sol3 = solve(exprs)
print(sol1)
print(sol2)

{K_st: 0.0514285714285714, K_stl: 0.0372413793103448, N_L: 0.950212931632136, N_P: 0.943918673394894, R: 1.01843016451792e-6, h: 14.2029991170233, h_c: 9.58084561156584, h_l: 3.82215350545746}
{K_st: 0.0514285714285714, K_stl: 0.0372413793103448, N_L: 0.950212931632136, N_P: 0.943918673394894, R: 1.01843016451792e-6, a: b + 1.0, h: 14.2029991170233, h_c: 9.58084561156584, h_l: 3.82215350545746}


### Scraping

In [ ]:
sc = Scraper()

### Pseudo Main

- choose DOI to scrape from
- use Scraper class to get MathML equations from paper and save to a txt file
- (also scrape tables for constants and save)
- LLMPredict class takes in MathML and constant files
- predicts python

